<a href="https://colab.research.google.com/github/edelangelmz/Inteligencia-Artificial/blob/main/PLN/English_Spanish_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from pickle import load

In [3]:
# Leer el conjunto de datos
drive.mount('/content/drive')
filename = '/content/drive/My Drive/Datasets/english-spanish.pkl'

dataset = load(open(filename, 'rb'))

Mounted at /content/drive


In [4]:
print(dataset.shape)
print(dataset[120000, 0])
print(dataset[120000, 1])

(124325, 2)
tom is a new yorker but he doesnt have a new york accent
tom es neoyorquino pero no tiene acento de nueva york


In [5]:
# Crear 'tokens'
source_tokens = []
for sentence in dataset[:, 0]:
  source_tokens.append(sentence.split(' '))

target_tokens = []
for sentence in dataset[:, 1]:
  target_tokens.append(sentence.split(' '))

print(source_tokens[120000])
print(target_tokens[120000])

['tom', 'is', 'a', 'new', 'yorker', 'but', 'he', 'doesnt', 'have', 'a', 'new', 'york', 'accent']
['tom', 'es', 'neoyorquino', 'pero', 'no', 'tiene', 'acento', 'de', 'nueva', 'york']


In [6]:
def build_token_dict(token_list):
  token_dict = {
      '<PAD>': 0,
      '<START>': 1,
      '<END>': 2
  }

  for tokens in token_list:
    for token in tokens:
      if token not in token_dict:
        token_dict[token] = len(token_dict)

  return token_dict

In [10]:
source_token_dict = build_token_dict(source_tokens)
target_token_dict = build_token_dict(target_tokens)
target_token_dict_inv = {v:k for k,v in target_token_dict.items()}

#for key, value in target_token_dict.items():
#  target_token_dict_inv[value] = key

print(list(source_token_dict.items())[:10])
print(list(target_token_dict.items())[:10])
print(list(target_token_dict_inv.items())[:10])

[('<PAD>', 0), ('<START>', 1), ('<END>', 2), ('go', 3), ('hi', 4), ('run', 5), ('who', 6), ('wow', 7), ('fire', 8), ('help', 9)]
[('<PAD>', 0), ('<START>', 1), ('<END>', 2), ('ve', 3), ('vete', 4), ('vaya', 5), ('vayase', 6), ('hola', 7), ('corre', 8), ('corran', 9)]
[(0, '<PAD>'), (1, '<START>'), (2, '<END>'), (3, 've'), (4, 'vete'), (5, 'vaya'), (6, 'vayase'), (7, 'hola'), (8, 'corre'), (9, 'corran')]


In [11]:
# Agregar <PAD>, <START> y <END> a cada frase del set de entrenamiento
encoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>'] for tokens in target_tokens]

#encoder_tokens = []
#for tokens in source_tokens:
#  encoder_tokens.append(['<START>'] + tokens + ['<END>'])

source_max_len = max(map(len, encoder_tokens))
target_max_len = max(map(len, decoder_tokens))

encoder_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encoder_tokens]
decoder_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decoder_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

In [12]:
print(encoder_tokens[120000])
print(decoder_tokens[120000])
print(output_tokens[120000])

['<START>', 'tom', 'is', 'a', 'new', 'yorker', 'but', 'he', 'doesnt', 'have', 'a', 'new', 'york', 'accent', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<START>', 'tom', 'es', 'neoyorquino', 'pero', 'no', 'tiene', 'acento', 'de', 'nueva', 'york', '<END>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['tom', 'es', 'neoyorquino', 'pero', 'no', 'tiene', 'acento', 'de', 'nueva', 'york', '<END>', '<PAD>', '<PAD>', '<PAD>', 

In [13]:
encoder_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encoder_tokens]
decoder_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decoder_tokens]
output_input = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

#encoder_input = []
#for tokens in encoder_tokens:
#  temp = []
#  for token in tokens:
#    temp.append(source_token_dict[token])
#  encoder_input.append(temp)

print(encoder_input[120000])
print(decoder_input[120000])
print(output_input[120000])

[1, 56, 258, 120, 197, 12666, 2914, 32, 1577, 140, 120, 197, 5385, 4287, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 123, 387, 23929, 1210, 45, 1006, 8223, 98, 1233, 10238, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[123], [387], [23929], [1210], [45], [1006], [8223], [98], [1233], [10238], [2], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]


In [14]:
pip install keras_transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-transformer: filename=keras_transformer-0.40.0-py3-none-any.whl size=12305 sha256=ae58b21278500fb913c71ac9d32b1461208303ea8dec1b69e59a37a77e45bd86
  Stored in directory: /root/.cache/pip/wheels/46/68/26/692ed21edd832833c3b0a0e21615bcacd99ca458b3f9ed571f
  Created wheel for keras-embed-sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3960 sha256=8531d84bf3bd944fc73d7792a973dfbe3d2d3ca6d0a700f81fe3d37a5cb1bee9
  Stored in directory: /root/.cache/pip/wheels/81/67/b5/d847588d075895281e1cf5590f819bd4cf076a554872268bd5
  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4668 sha256=ad0e1ff9f7a7a85924b3245ddf97fe76aa3b6268ae2f4b7a2ab9950c3c3005af
  Stored in directory: /root/.cache/pip/wheels/85/5d/1c/2e619f594f69fbcf8bc20943b27d414871c409be053994813e
  Created wheel for keras-multi-head: filename=kera

In [15]:
# Crear la red neuronal
from keras_transformer import get_model, decode
import numpy as np

# Establecer una semilla inicial
np.random.seed(0)

In [16]:
model = get_model(
    token_num = max(len(source_token_dict), len(target_token_dict)),
    embed_dim = 32,
    encoder_num = 2,
    decoder_num = 2,
    head_num = 4,
    hidden_dim = 128,
    dropout_rate = 0.05,
    use_same_embed = False
)

model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Encoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Encoder-Token-Embedding (Embed  [(None, None, 32),  808608      ['Encoder-Input[0][0]']          
 dingRet)                        (25269, 32)]                                                     
                                                                                                  
 Encoder-Embedding (TrigPosEmbe  (None, None, 32)    0           ['Encoder-Token-Embedding[0][0]']
 dding)                                                                                           
                                                                                              

In [17]:
# Entrenamiento
x = [np.array(encoder_input), np.array(decoder_input)]
y = np.array(output_input)

filename = "/content/drive/My Drive/Models/english-spanish.h5"

#model.fit(x, y, epochs = 15, batch_size = 32)
#model.save(filename)

model.load_weights(filename)

In [18]:
def translate(sentence):
  sentence_tokens = [['<START>'] + tokens + ['<END>', '<PAD>'] for tokens in [sentence.split(' ')]]
  #sentence_tokens = [tokens + ['<END>', '<PAD>'] for tokens in [sentence.split(' ')]]
  
  tr_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in sentence_tokens][0]
  decoded = decode(
      model,
      tr_input,
      start_token = target_token_dict['<START>'],
      end_token = target_token_dict['<END>'],
      pad_token = target_token_dict['<PAD>']
  )

  print('Traducción: {}'.format(' '.join(map(lambda x: target_token_dict_inv[x], decoded[1:-1]))))

In [19]:
translate('hot shower')
translate('the cat ate the whole food')

Traducción: se ducha
Traducción: el gato se comio la comida entera
